* run a hyperparameter sweep on a training script
* compare the results of the runs in the MLFlow UI
* choose the best run and register it as a model
* deploy the model to a REST API
* build a container image suitable for deployment to a cloud platform


In [1]:
import pandas as pd
import keras
import numpy as np
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe # for hyperparameter tuning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import mlflow
from mlflow.models import infer_signature



In [2]:
# load the dataset
data=pd.read_csv("./winequality-white.csv",sep=";")
data


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
# split the data into training, validation and test sets

train,test=train_test_split(data,test_size=0.25,random_state=42)


In [4]:
train_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()

#test dataset
test_x=test.drop(['quality'],axis=1).values
test_y=test[['quality']].values.ravel()

#splitting the train data into train and validation
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature=infer_signature(train_x,train_y)


In [5]:
# ANN model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    # define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    #compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
        )
    
    #train the ANN model with lr and momentum params with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        ## evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)
        eval_rmse=eval_result[1]

        #log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        #log the model
        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return{"loss":eval_rmse,"status":STATUS_OK,"model":model}



In [6]:
def objective(params):
    # mlflow will track the parametes and results for each run
    result=train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result


In [7]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [9]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    #conduct the hyperparameter seach using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

# fetch the details of the best run
best_run=sorted(trials.results,key=lambda x:x["loss"])[0]


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 15s 339ms/step - loss: 35.2514 - root_mean_squared_error: 5.3564
 2/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.9635 - root_mean_squared_error: 5.0939
 3/46 ━━━━━━━━━━━━━━━━━━━━ 0s 668us/step - loss: 28.9150 - root_mean_squared_error: 5.0939
 5/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 24.1858 - root_mean_squared_error: 4.8660
 4/46 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - loss: 26.3230 - root_mean_squared_error: 5.0939
 6/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.4720 - root_mean_squared_error: 4.6765
 7/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.0125 - root_mean_squared_error: 4.5085   
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5289 - root_mean_squared_error: 2.5739 - val_loss: 1.0374 - val_root_mean_squared_error: 1.0185

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.9587 - root_mean_squared_error: 0.9792
 2/46 ━━━━━━━━━━━━━━━━

In [12]:
#inference

from mlflow.models import validate_serving_input

model_uri = 'runs:/ab758e8aa5794998a41a9e03f4f62dfa/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\Sandesh\Desktop\sandesh\MLFlowStarter\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.9821033],
       [6.3276596],
       [6.3030214],
       ...,
       [5.9219046],
       [6.6190743],
       [5.82569  ]], dtype=float32)

In [14]:
# load model as a pyfunc model
model_uri = 'runs:/ab758e8aa5794998a41a9e03f4f62dfa/model'

loaded_model=mlflow.pyfunc.load_model(model_uri)

#predict on pandas dataframe
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step


array([[5.9821033],
       [6.3276596],
       [6.3030214],
       ...,
       [5.9219046],
       [6.6190743],
       [5.82569  ]], dtype=float32)

In [15]:
## register in the model registry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1729306885389, current_stage='None', description=None, last_updated_timestamp=1729306885389, name='wine-quality', run_id='ab758e8aa5794998a41a9e03f4f62dfa', run_link=None, source='file:///c:/Users/Sandesh/Desktop/sandesh/MLFlowStarter/2-DLproject/mlruns/205197789399967776/ab758e8aa5794998a41a9e03f4f62dfa/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>